In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

### Standard imports

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Important for multiprocessing.
import torch
torch.set_num_threads(1)

### Lattice imports

In [ ]:
from collections import OrderedDict
from gridsearch import experiment, load_experiment

# Create NARMA dataset

In [ ]:
import dataset as ds

u_train, y_train = ds.NARMA(sample_len = 2000)
u_test, y_test = ds.NARMA(sample_len = 3000)
dataset = [u_train, y_train, u_test, y_test]
ds.dataset = dataset

# Waxman graph generation

In [ ]:
from matrix import waxman
from plot import scatter_3d

In [ ]:
from matrix import euclidean

# We need names for the distance functions to select them from a pandas
# dataframe later.
euc = euclidean
def inv(x, y): return 1/euclidean(x, y)
def inv_squared(x, y): return 1/euclidean(x, y)**2

### Waxman visualization

In [ ]:
for z_frac in [0.0, 0.5, 1.0]:
    G = waxman(n=200, alpha=1.0, beta=1.0, z_frac=z_frac)
    scatter_3d(G)

### Waxman performance with increasing fraction of 3d nodes

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['hidden_nodes'] = np.arange(20, 90, 10)
params['z_frac'] = np.arange(0.0, 1.2, 0.2)
params['directed'] = [True, False]
waxman_nrmse_df = experiment(esn_nrmse, params)
waxman_nrmse_df.to_pickle('experiments/waxman_nrmse.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_nrmse_df' not in locals():
    waxman_nrmse_df = load_experiment('experiments/waxman_nrmse.pkl')

undirected_df = waxman_nrmse_df.loc[waxman_nrmse_df['directed'] == False]
directed_df = waxman_nrmse_df.loc[waxman_nrmse_df['directed'] == True]

zlim=(0.5, 1.0)
azim=130

plot_df_trisurf(df=undirected_df,
                groupby=['hidden_nodes', 'z_frac'],
                axes=['hidden_nodes', 'z_frac', 'esn_nrmse'],
                azim=azim, zlim=zlim, title='Undirected')
plot_df_trisurf(df=directed_df,
                groupby=['hidden_nodes', 'z_frac'],
                axes=['hidden_nodes', 'z_frac', 'esn_nrmse'],
                azim=azim, zlim=zlim, title='Directed')

There does not seem to be a difference when changing from undirected to directed  
graphs, which I attribute to the fact that I think that both methods are  
saturating the activations of hidden nodes anyway, so it does not matter.  

_Afterthought_ for the above: isn't the spectral radius dealing with that  
anyway? It is probably necessary to actually probe the network to tell for sure.  

The hypothesis that nodes are being saturated are backed up by the fact that the  
error _decreases_ as the networks become smaller, indicating that we are  
reaching a point where nodes may begin to desaturate.  

### Distribution of reservoir weights

In [ ]:
from ESN import ESN
from plot import plot_esn_weight_hist

bins = 40
params = {
    'hidden_nodes': 50,
}

plt.title('Reference ESN')
plot_esn_weight_hist(params, n_bins=bins, show=True)

old_figsize = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = (14, 6)

params = {
    'directed': False,
    'w_res_type': 'waxman',
}

# z_frac = 0.0.
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
plt.suptitle('z_frac: 0.0')
params['dist_function'] = euc
params['z_frac'] = 0.0

params['hidden_nodes'] = 20
ax1.set_title('hidden_nodes: ' + str(params['hidden_nodes']))
plot_esn_weight_hist(params, n_bins=bins, ax=ax1, show=False)
params['hidden_nodes'] = 50
ax2.set_title('hidden_nodes: ' + str(params['hidden_nodes']))
plot_esn_weight_hist(params, n_bins=bins, ax=ax2, show=False)
params['dist_function'] = inv
ax3.set_title('dist_function: ' + params['dist_function'].__name__)
plot_esn_weight_hist(params, n_bins=bins, ax=ax3, show=False)
params['dist_function'] = inv_squared
ax4.set_title('dist_function: ' + params['dist_function'].__name__)
plot_esn_weight_hist(params, n_bins=bins, ax=ax4, show=True)

# z_frac = 1.0.
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
plt.suptitle('z_frac: 1.0')
params['dist_function'] = euc
params['z_frac'] = 1.0

params['hidden_nodes'] = 20
ax1.set_title('hidden_nodes: ' + str(params['hidden_nodes']))
plot_esn_weight_hist(params, n_bins=bins, ax=ax1, show=False)
params['hidden_nodes'] = 50
ax2.set_title('hidden_nodes: ' + str(params['hidden_nodes']))
plot_esn_weight_hist(params, n_bins=bins, ax=ax2, show=False)
params['dist_function'] = inv
ax3.set_title('dist_function: ' + params['dist_function'].__name__)
plot_esn_weight_hist(params, n_bins=bins, ax=ax3, show=False)
params['dist_function'] = inv_squared
ax4.set_title('dist_function: ' + params['dist_function'].__name__)
plot_esn_weight_hist(params, n_bins=bins, ax=ax4, show=True)

plt.rcParams["figure.figsize"] = old_figsize

We see very clearly that the weight distributions are tightened when we increase  
the number of nodes of the network (changing from leftmost to second to leftmost  
plot), which is caused by the spectral radius scaling to be stricter. However,  
this is just scaling by a scalar value.  

When the weight function moves from d to 1/d to 1/d, we see that we are shifting  
the probability distribution towards many lower weights, with just a few big  
ones, which is as expected.  

### Waxman performance with increasing fraction of negative weights

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['z_frac'] = [1.0]
params['hidden_nodes'] = np.arange(20, 90, 10)
params['sign_frac'] = np.arange(0.0, 0.55, 0.05)
params['directed'] = [True, False]
waxman_sign_df = experiment(esn_nrmse, params)
waxman_sign_df.to_pickle('experiments/waxman_sign.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_sign_df' not in locals():
    waxman_sign_df = load_experiment('experiments/waxman_sign.pkl')

undirected_df = waxman_sign_df.loc[waxman_sign_df['directed'] == False]
directed_df = waxman_sign_df.loc[waxman_sign_df['directed'] == True]

azim=70
zlim=(0.3, 1.0)

plot_df_trisurf(df=undirected_df,
                groupby=['hidden_nodes', 'sign_frac'],
                axes=['hidden_nodes', 'sign_frac', 'esn_nrmse'],
                azim=azim, zlim=zlim, title='Undirected')
plot_df_trisurf(df=directed_df,
                groupby=['hidden_nodes', 'sign_frac'],
                axes=['hidden_nodes', 'sign_frac', 'esn_nrmse'],
                azim=azim, zlim=zlim, title='Directed')

We see what happens with the undirected (as we think this to be the most  
relevant for physical computing) type of graph in three dimensions, as the  
previous results show little difference between 2 and 3 dimensions.  

There is a noticeable difference between the directed and undirected graphs  
here. Dale et al. investigates this a bit in their paper: «It then becomes clear  
that how weights are struc- tured and directed, controlling information flow,  
has a greater affect on quality of the network. This supports similar results  
using hierarchical networks, where structure and number of parameters also  
significantly impact performance [6].»  

### Waxman performance with changing weight/distance function

In [ ]:
# %%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['z_frac'] = np.arange(0.0, 1.2, 0.2)
params['hidden_nodes'] = np.arange(20, 90, 10)
params['dist_function'] = [euc, inv, inv_squared]
waxman_dist_df = experiment(esn_nrmse, params, runs=20)
waxman_dist_df.to_pickle('experiments/waxman_dist.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_dist_df' not in locals():
    waxman_dist_df = load_experiment('experiments/waxman_dist.pkl')

waxman_dist_df['dist_function'] = waxman_dist_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

euc_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == euc.__name__]
inv_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == inv.__name__]
inv_squared_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == inv_squared.__name__]

zlim=(0.3, 1.0)
azim=110

for df in [(euc_df, 'd'), (inv_df, '1/d'), (inv_squared_df, '1/d^2')]:
    fig = plt.figure(figsize=plt.figaspect(0.4))
    ax1 = plt.subplot(1, 2, 1, projection='3d')
    ax2 = plt.subplot(1, 2, 2, projection='3d')
    axs = [ax1, ax2]

    fig.suptitle(df[1])
    for i, agg in enumerate(['mean', 'min']):
        plot_df_trisurf(df=df[0],
                        groupby=['hidden_nodes', 'z_frac'],
                        axes=['hidden_nodes', 'z_frac', 'esn_nrmse'],
                        agg=agg,
                        azim=azim, zlim=zlim,
                        title=agg,
                        ax=axs[i], show=False)

    plt.tight_layout()
    plt.show()

From a short preliminary study: it does not seem like different distance  
functions such as d, 1/d, 1/d^2 and 1/d^3 make much of a difference at all. This  
was tested both with what's shown above, as well as manually with increased size  
of the waxman volume (to ensure that we don't have a majority of <1 elements).  

Another interesting thing to plot is the probability distribution of the weights  
to compare between the dimensionalities.  

# Lattice/tiling experiments (sq, rect, hex, tri)

### Plots of lattices

In [ ]:
from ESN import ESN
from plot import plot_lattice

old_figsize = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = (14, 6)

esn_square = ESN(hidden_nodes=25, w_res_type='tetragonal')
esn_hex = ESN(hidden_nodes=25, w_res_type='hexagonal')
esn_tri = ESN(hidden_nodes=25, w_res_type='triangular')
esn_rect = ESN(hidden_nodes=25, w_res_type='rectangular', rect_ratio=2.0)

G_square = esn_square.G
G_hex = esn_hex.G
G_tri = esn_tri.G
G_rect = esn_rect.G

fig, axs = plt.subplots(2, 2)
ax1, ax2, ax3, ax4 = axs[0, 0], axs[0, 1], axs[1, 0], axs[1, 1]
plot_lattice(G_square, title='Square', ax=ax1, show=False)
plot_lattice(G_hex, title='Hexagonal', ax=ax2, show=False)
plot_lattice(G_tri, title='Triangular', ax=ax3, show=False)
plot_lattice(G_rect, title='Rectangular', ax=ax4, show=True)

plt.rcParams["figure.figsize"] = old_figsize

### Lattice NRMSE

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
nrmse_df = experiment(esn_nrmse, params)
nrmse_df.to_pickle('experiments/lattice_nrmse.pkl')

In [ ]:
if 'nrmse_df' not in locals():
    nrmse_df = load_experiment('experiments/lattice_nrmse.pkl')

grouped_df = nrmse_df.groupby(['hidden_nodes', 'w_res_type']).mean().reset_index()

tetragonal = grouped_df.loc[grouped_df['w_res_type'] == 'tetragonal']
hexagonal = grouped_df.loc[grouped_df['w_res_type'] == 'hexagonal']
triangular = grouped_df.loc[grouped_df['w_res_type'] == 'triangular']

plt.plot(tetragonal['hidden_nodes'], tetragonal['esn_nrmse'], label='sq')
plt.plot(hexagonal['hidden_nodes'], hexagonal['esn_nrmse'], label='hex')
plt.plot(triangular['hidden_nodes'], triangular['esn_nrmse'], label='tri')

plt.legend(fancybox=False, loc='upper right', bbox_to_anchor=(1.0, 1.0))
plt.ylabel('NRMSE')
plt.xlabel('Hidden nodes')

plt.show()

### Lattice STM

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
mc_df = experiment(esn_mc, params)
mc_df.to_pickle('experiments/lattice_mc.pkl')

In [ ]:
if 'mc_df' not in locals():
    mc_df = load_experiment('experiments/lattice_mc.pkl')

grouped_df = mc_df.groupby(['hidden_nodes', 'w_res_type']).mean().reset_index()

tetragonal = grouped_df.loc[grouped_df['w_res_type'] == 'tetragonal']
hexagonal = grouped_df.loc[grouped_df['w_res_type'] == 'hexagonal']
triangular = grouped_df.loc[grouped_df['w_res_type'] == 'triangular']

plt.plot(tetragonal['hidden_nodes'], tetragonal['esn_mc'], label='sq')
plt.plot(hexagonal['hidden_nodes'], hexagonal['esn_mc'], label='hex')
plt.plot(triangular['hidden_nodes'], triangular['esn_mc'], label='tri')

plt.legend(fancybox=False, loc='upper right', bbox_to_anchor=(1.0, 1.0))
plt.ylabel('MC')
plt.xlabel('Hidden nodes')

plt.show()

### Lattice input scaling

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['input_scaling'] = np.arange(0.1, 2.1, 0.1)
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
is_df = experiment(esn_mc, params)
is_df.to_pickle('experiments/lattice_input_scaling.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'is_df' not in locals():
    is_df = load_experiment('experiments/lattice_input_scaling.pkl')

tetragonal = is_df.loc[is_df['w_res_type'] == 'tetragonal']
hexagonal = is_df.loc[is_df['w_res_type'] == 'hexagonal']
triangular = is_df.loc[is_df['w_res_type'] == 'triangular']

labels = {'x': 'Input scaling', 'y': 'Hidden nodes', 'z': 'MC'}
zlim = (5, 15)

plot_df_trisurf(df=tetragonal, groupby=['hidden_nodes', 'input_scaling'],
                axes=['input_scaling', 'hidden_nodes', 'esn_mc'],
                labels=labels, zlim=zlim)

plot_df_trisurf(df=hexagonal, groupby=['hidden_nodes', 'input_scaling'],
                axes=['input_scaling', 'hidden_nodes', 'esn_mc'],
                labels=labels, zlim=zlim)

plot_df_trisurf(df=triangular, groupby=['hidden_nodes', 'input_scaling'],
                axes=['input_scaling', 'hidden_nodes', 'esn_mc'],
                labels=labels, zlim=zlim)

### Using a rectangular lattice instead of square

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['w_res_type'] = ['rectangular']
params['rect_ratio'] = np.arange(0.1, 3.1, 0.1)
rect_df = experiment(esn_nrmse, params)
rect_df.to_pickle('experiments/lattice_rect.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'rect_df' not in locals():
    rect_df = load_experiment('experiments/lattice_rect.pkl')

azim=40
zlim=(0.4, 0.7)

plot_df_trisurf(df=rect_df,
                groupby=['hidden_nodes', 'rect_ratio'],
                axes=['hidden_nodes', 'rect_ratio', 'esn_nrmse'],
                azim=azim, zlim=zlim, title='Effect of rectangular lattice')

### Periodic lattice

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['periodic'] = [True, False]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
periodic_df = experiment(esn_mc, params)
periodic_df.to_pickle('experiments/periodic_lattice.pkl')

In [ ]:
if 'periodic_df' not in locals():
    periodic_df = load_experiment('experiments/periodic_lattice.pkl')

grouped_df = periodic_df.groupby(['hidden_nodes', 'periodic', 'w_res_type']).mean().reset_index()

tetragonal = grouped_df.loc[(grouped_df['w_res_type'] == 'tetragonal') & (grouped_df['periodic'] == False)]
tetragonal_periodic = grouped_df.loc[(grouped_df['w_res_type'] == 'tetragonal') & (grouped_df['periodic'] == True)]

hexagonal = grouped_df.loc[(grouped_df['w_res_type'] == 'hexagonal') & (grouped_df['periodic'] == False)]
hexagonal_periodic = grouped_df.loc[(grouped_df['w_res_type'] == 'hexagonal') & (grouped_df['periodic'] == True)]

triangular = grouped_df.loc[(grouped_df['w_res_type'] == 'triangular') & (grouped_df['periodic'] == False)]
triangular_periodic = grouped_df.loc[(grouped_df['w_res_type'] == 'triangular') & (grouped_df['periodic'] == True)]

plt.title('Periodic lattices')
plt.ylabel('MC')
plt.xlabel('Hidden nodes')

plt.plot(tetragonal['hidden_nodes'], tetragonal['esn_mc'], label='Tetragonal non-periodic', color='green')
plt.plot(tetragonal_periodic['hidden_nodes'], tetragonal_periodic['esn_mc'], label='Tetragonal periodic', linestyle='--', color='green')

plt.plot(hexagonal['hidden_nodes'], hexagonal['esn_mc'], label='Hexagonal non-periodic', color='red')
plt.plot(hexagonal_periodic['hidden_nodes'], hexagonal_periodic['esn_mc'], label='Hexagonal periodic', linestyle='--', color='red')

plt.plot(triangular['hidden_nodes'], triangular['esn_mc'], label='Triangular non-periodic', color='blue')
plt.plot(triangular_periodic['hidden_nodes'], triangular_periodic['esn_mc'], label='Triangular periodic', linestyle='--', color='blue')

plt.legend(fancybox=False, loc='lower right', bbox_to_anchor=(1.0, 0.0))

plt.show()